In [ ]:
pip install ultralytics -qq
pip install -U tensorflow-addons -qq



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2, glob, os, random
import warnings
warnings.filterwarnings('ignore')

from ultralytics import YOLO
from skimage.io import imshow
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Nadam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow_addons as tfa

# ignore warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))

        cv2.rectangle(
            image,
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.png'))
    all_images.extend(glob.glob(image_paths+'/*.PNG'))

    all_images.sort()

    num_images = len(all_images)

    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.strip().split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

In [ ]:
plot(image_paths='/kaggle/input/plate-detection-utils/data/train/images',
     label_paths='/kaggle/input/plate-detection-utils/data/train/labels',
     num_samples=4)

In [ ]:
%%writefile data.yaml
path: '/kaggle/input/plate-detection-utils/data'
train: 'train/images'
val: 'valid/images'

nc: 16
names: ['dog', 'person', 'cat', 'tv', 'car', 'meatballs', 'marinara sauce', 'tomato soup', 'chicken noodle soup', 'french onion soup', 'chicken breast', 'ribs', 'pulled pork', 'hamburger', 'cavity', 'char']
# will only be using char class

In [ ]:
# EPOCH = 50
# IMG_SIZE = 320

# model = YOLO('yolov8n.pt')
# model.train(data='data.yaml', epochs=EPOCH, imgsz=IMG_SIZE, name='yolov8n_v8_50e')

In [ ]:
model = YOLO('/kaggle/input/plate-detection-utils/data/best.pt')
metrics = model.val()
metrics.box.map

In [ ]:
source = '/kaggle/input/test-images/test'
model.predict(source, imgsz=320, conf=0.5, name='yolov8n_v8_50e_testpred', save_txt=True, save_crop=True)

In [ ]:
plot(
    image_paths='/kaggle/input/test-images/test',
    label_paths='/kaggle/working/runs/detect/yolov8n_v8_50e_testpred/labels',
    num_samples=4
)

In [ ]:
def segment(img_path, txt_path):
    image = cv2.imread(img_path)
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    bounding_box_images = []
    for line in lines:
        line = line.strip().split()
        label = line[0]
        x, y, w, h = map(float, line[1:])
        left = int((x - w / 2) * image.shape[1])
        top = int((y - h / 2) * image.shape[0])
        right = int((x + w / 2) * image.shape[1])
        bottom = int((y + h / 2) * image.shape[0])

        bbox_image = image[top:bottom, left:right]
        bounding_box_images.append((left, bbox_image))

    # Sort the bounding boxes based on their left coordinate
    bounding_box_images.sort(key=lambda x: x[0])

    # Extract the images in left-to-right order
    bounding_box_images = [bbox_image for _, bbox_image in bounding_box_images]

    return bounding_box_images


In [ ]:
IMG_PATH = '/kaggle/input/test-images/test/DataTest82.png'
TXT_PATH = '/kaggle/working/runs/detect/yolov8n_v8_50e_testpred/labels/DataTest82.txt'

seg_img = segment(IMG_PATH, TXT_PATH)

In [ ]:
chars = []
for char in seg_img:
  chars.append(cv2.resize(char, (24, 43)))

In [ ]:
for i in range(len(chars)):
  plt.subplot(1, len(chars), i+1)
  plt.imshow(chars[i], cmap='gray')
  plt.axis('off')
plt.show()

In [ ]:
def format_img(img_arr):
  gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
#   blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#   sharpened = cv2.addWeighted(gray, 1.5, blurred, -0.5, 0)
#   _, black_white = cv2.threshold(sharpened, 127, 255, cv2.THRESH_BINARY)

  return gray

In [ ]:
chars = [format_img(char) for char in chars]

for i in range(len(chars)):
  plt.subplot(1, len(chars), i+1)
  plt.imshow(chars[i], cmap='gray')
  plt.axis('off')
plt.show()

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1)
path = '/kaggle/input/plate-detection-utils'

train_gen = datagen.flow_from_directory(
    os.path.join(path, 'train_img'),
    target_size=(32,32),
    batch_size=1,
    class_mode='categorical'
)
val_gen = datagen.flow_from_directory(
    os.path.join(path, 'val_img'),
    target_size=(32,32),
    batch_size=1,
    class_mode='categorical'
)

In [ ]:
# callbacks
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', 
                              factor = 0.3, 
                              patience = 2, 
                              min_delta = 0.001, 
                              mode = 'min', verbose = 1)
model_save = ModelCheckpoint('./LicensePlateModelv2_best_weights.h5',
                            save_best_only=True,
                            save_weights_only=True,
                            monitor='val_loss',
                            mode='min',
                            verbose=0)
# early_stopping = EarlyStopping(monitor='val_loss',
#                               min_delta=0.001,
#                               patience=8,
#                               mode='min',
#                               verbose=0,
#                               restore_best_weights=True)
tqdm = tfa.callbacks.TQDMProgressBar()

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))

model.add(Dense(36,activation ="softmax"))

In [ ]:
model.summary()

In [ ]:
optimizers = [
    tf.keras.optimizers.SGD(learning_rate=0.001),
    tf.keras.optimizers.Adam(learning_rate=0.001),
    tf.keras.optimizers.RMSprop(learning_rate=0.001),
    tf.keras.optimizers.Adagrad(learning_rate=0.01),
    tf.keras.optimizers.Nadam(learning_rate=0.001)
    # It'll take an eternity if I include more optimizers, lol
]

In [ ]:
results = []

for optimizer in optimizers:
    # Compile the model with the current optimizer
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['acc']
    )

    # Train the model
    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // 1,
        validation_data=val_gen,
        epochs=80,
        verbose=0,
        callbacks=[tqdm, reduce_lr, model_save]
    )

    # Store the history for this optimizer
    results.append({
        'optimizer': optimizer.__class__.__name__,
        'history': history
    })

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

for result in results:
    history_df = pd.DataFrame(result['history'].history)
    
    axs[0].set_title('Validation Loss')
    axs[0].plot(history_df.loc[:, ['loss', 'val_loss']], label=result['optimizer'])

    axs[1].set_title('Accuracy')
    axs[1].plot(history_df.loc[:, ['acc', 'val_acc']], label=result['optimizer'])


In [ ]:

plt.legend()
plt.show()

In [ ]:
def create_model():
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Flatten())

    model.add(Dense(64,activation ="relu"))
    model.add(Dense(128,activation ="relu"))

    model.add(Dense(36,activation ="softmax"))
    return model

from keras.models import load_model
model_loaded = create_model()
model_loaded.load_weights('LicensePlateModelv2_best_weights.h5')

In [ ]:
def fix_dimension(img):
  new_img = np.zeros((32,32,3))
  for i in range(3):
    new_img[:,:,i] = img
  return new_img

def show_results():
  dic = {}
  characters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
  for i, c in enumerate(characters):
    dic[i] = c

  output = []
  for i, ch in enumerate(chars):
    img = cv2.resize(ch, (32,32), interpolation=cv2.INTER_AREA)
    img = fix_dimension(img)
    img = img.reshape(1,32,32,3)
    y = model_loaded.predict(img)
    pred_class = np.argmax(y, axis=-1)[0]
    output.append(dic[pred_class])
    
  plate_number = ''.join(output)
  return plate_number
        

print(show_results())

In [ ]:
image_files = glob.glob('/kaggle/input/test-images/test/*.png')    
label_files = glob.glob('/kaggle/working/runs/detect/yolov8n_v8_50e_testpred/labels/*.txt')

image_names = [os.path.splitext(os.path.basename(file))[0] for file in image_files]
label_names = [os.path.splitext(os.path.basename(file))[0] for file in label_files]

# sort lists
image_names = sorted(image_names, key=lambda x: int(x[8:]))
label_names = sorted(label_names, key=lambda x: int(x[8:]))

image_path = '/kaggle/input/test-images/test'
label_path = '/kaggle/working/runs/detect/yolov8n_v8_50e_testpred/labels'
seg_img = 0
results = []
for image_name in image_names:
    for label_name in label_names:
        if image_name == label_name:
            print(f"File: {image_name}")
            seg_img = segment(os.path.join(image_path, image_name + '.png'), os.path.join(label_path, label_name + '.txt'))
            chars = []
            for char in seg_img:
              chars.append(cv2.resize(char, (24, 43)))
            chars = [format_img(char) for char in chars]
            
            results.append(show_results())
            
            break

In [ ]:
results

In [ ]:
# convert to dataframe format
import pandas as pd
df = pd.DataFrame({'Name of File': image_names,
                  'Vehicleregistrationplatebymodel': results})
df.head()

In [ ]:
# df.to_csv('submission5PerfectlyBalanced.csv')